In [1]:
!pip install chromadb

In [2]:
!pip install -U langchain-chroma

In [3]:
import os
import pysqlite3
import sys
sys.modules["sqlite3"] = sys.modules.pop("pysqlite3")
import chromadb
from chromadb.config import Settings

In [4]:
import gradio as gr
from llama_index.llms.ollama import Ollama
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.llms.base import LLM

from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
#from langchain_community.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.vectorstores import VectorStore
from typing import List, Any
from chromadb import Client
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain

/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
chroma_client = chromadb.Client()

In [6]:
# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="dwarf_collect")

In [7]:
# Define a custom LangChain wrapper for the Ollama LLaMA model
class OllamaLangChain(LLM):
    model_name: str = "llama3.2:latest"  # Declared field
    request_timeout: int = 90           # Declared field

    def __init__(self, model_name="llama3.2:latest", request_timeout=30):
        super().__init__()
        self.model_name = model_name
        self.request_timeout = request_timeout
        # Use object.__setattr__ to bypass Pydantic's field validation
        object.__setattr__(self, "_client", Ollama(model=model_name, request_timeout=request_timeout))
    
    @property
    def _llm_type(self):
        return "ollama"

    def _call(self, prompt, stop=None):
        # Access the private client
        client = object.__getattribute__(self, "_client")
        response = client.complete(prompt)
        return response.text

In [8]:
# Read the text file
with open("docs/dwarf.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Create a list of a single document
documents = [Document(page_content=text)]

In [9]:
embeddings = OllamaEmbeddings(
    model="llama3.2"
)

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 200,
    chunk_overlap  = 16
    
)
# Split the text into chunks
splitted_txt = text_splitter.split_documents(documents)

In [11]:
# Output the result
for chunk in splitted_txt:
    print(chunk.page_content)
    print("\n")

Dwarf
Dwarf Traits
Your dwarf character has an assortment of inborn
abilities, part and parcel of dwarven nature.
Ability Score Increase. Your Constitution score
increases by 2.


increases by 2.
Age. Dwarves mature at the same rate as humans,
but they’re considered young until they reach the
age of 50. On average, they live about 350 years.


Alignment. Most dwarves are lawful, believing
firmly in the benefits of a well-­‐‑ordered society. They
tend toward good as well, with a strong sense of fair


play and a belief that everyone deserves to share in
the benefits of a just order.
Size. Dwarves stand between 4 and 5 feet tall and
average about 150 pounds. Your size is Medium.


Speed. Your base walking speed is 25 feet. Your
speed is not reduced by wearing heavy armor.
Darkvision. Accustomed to life underground, you
have superior vision in dark and dim conditions. You


can see in dim light within 60 feet of you as if it were
bright light, and in darkness as if it were dim light.
You 

In [12]:
vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="dwarf_collect",
    client=chroma_client
)


In [13]:
# Add documents to the collection
vectorstore.add_documents(splitted_txt)

['3bc6279b-fee2-4fa4-bc2a-9c489578f502',
 '047d205e-403a-4a7c-a0ed-a22544ef1fc5',
 'dac901d1-2d37-493e-811a-506cf879d5ca',
 '78297034-e183-4816-a5be-a768ff5fcab3',
 'c252346d-67cf-419b-9ee1-03ccd039cd04',
 'ea0dd699-5b80-4296-9ef5-314d39fdaba2',
 '49eaea9a-0292-483b-87c5-aa106c666ee3',
 '7ad0c210-8e54-4a5d-87b2-ce18a1d4ba63',
 '0db687ce-d29e-4daa-849d-8da333f09ee5',
 '4f73b01f-a334-484e-9d76-596dee2bacf7',
 '85455c5e-466e-4042-9352-1af052561ff4',
 '233dc1ee-f794-4da4-af4d-1c4cd57533a6',
 '65792cb0-58a2-47bc-ad05-b139e990be24']

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
llm = OllamaLangChain()
memory = ConversationBufferMemory()  # Store chat history


In [16]:
conversation = ConversationChain(llm=llm, memory=memory, verbose=True)

/tmp/ipykernel_264347/3355408383.py:1: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation = ConversationChain(llm=llm, memory=memory, verbose=True)


In [17]:
prompt_template = PromptTemplate(
    #input_variables=["question", "context"],  # context here is the document content
    input_variable=["questions"],
    template="Given the context:, answer the question: {question}"
)
# Create the LLMChain for the question generation
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    verbose=True
)

/tmp/ipykernel_264347/3409293799.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(


In [18]:
# Define the combine_docs_chain with document_variable_name 'context'
combine_docs_chain = load_qa_chain(
    llm=llm,
    chain_type='stuff',
    #document_variable_name='context',  # Ensure this matches the input in the LLMChain
    verbose=True
)

# Create the ConversationalRetrievalChain with the manually created question generator chain
retrieval_chain = ConversationalRetrievalChain(
    retriever=retriever,  # Your retriever instance
    question_generator=llm_chain,  # Use the LLMChain as question generator
    combine_docs_chain=combine_docs_chain,  # Chain to combine the documents
    memory=memory,  # Memory to store conversation history
    verbose=True  # Enable verbosity for debugging
)

/tmp/ipykernel_264347/368744455.py:2: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  combine_docs_chain = load_qa_chain(
/tmp/ipykernel_264347/368744455.py:10: LangChainDeprecationWarning: The class `ConversationalRetrievalChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use create_history_aware_retriever together with create_retrieval_chain (see example in docstring) instead.
  retrieval_chain = Conversa

In [19]:
# Chat function
"""def chat_with_langchain(input_text):
    # Use LangChain's ConversationChain
    response = conversation.run(input_text)
    return response"""

"def chat_with_langchain(input_text):\n    # Use LangChain's ConversationChain\n    response = conversation.run(input_text)\n    return response"

In [20]:
# Define a chat function for Gradio
def chat_with_langchain(input_text):
    # Check if the input contains any document-related query
    
    # Retrieve relevant documents based on input query
    context = retriever.get_relevant_documents(input_text)  # Get the relevant document context
    question = input_text  # Use the entire input as the question
    
    # Ensure you only pass the 'question' key to the LLM chain
    #response = llm_chain.run({"question": question, "context": context})
    response = llm_chain.run({"question": question})
    return response




In [21]:
# Gradio interface
interface = gr.Interface(
    fn=chat_with_langchain,
    inputs="text",
    outputs="text",
    title="LLaMA 3 Chatbot with LangChain",
    description="Chat with a LLaMA 3-based model via Ollama and LangChain!"
)
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://da0ffb875aa88739dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/tmp/ipykernel_264347/1367368394.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  context = retriever.get_relevant_documents(input_text)  # Get the relevant document context
/tmp/ipykernel_264347/1367368394.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = llm_chain.run({"question": question})
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Given the context:, answer the question: I am a Dwarf, Tell me about Size ?

> Finished chain.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Given the context:, answer the question: I am a Dwarf, Tell me about weight ?

> Finished chain.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
Given the context:, answer the question: I am a Dwarf, Tell me about Alignment ?


Traceback (most recent call last):
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/httpx/_transports/default.py", line 69, in map_httpcore_exceptions
    yield
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/httpx/_transports/default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 196, in handle_request
    response = connection.handle_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/capmars/SemAgent/.venv/lib/python3.11/site-packages/httpcore/_sync/connection.py", line 101, in handle_request
    return self._connection.handle_request(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ca